In [85]:
from functions import *

In [86]:
df = load_airtable_data()
df.to_csv('Data/airtable_data.csv', index=False)

In [143]:
df = pd.read_csv('Data/airtable_preprocessed_data.csv')

In [155]:
import pandas as pd
import numpy as np
from datetime import datetime

def success_rate(df):
    """
    Calculates the success rate of ticket prices for events based on ticket prices and remaining prices.

    The success rate is calculated as: 1 - (remain price / ticket price). 
    Values close to 1 indicate a high success rate.

    Args:
        df (pd.DataFrame): DataFrame containing 'ticket_price' and 'remain_prices' columns.

    Returns:
        list: A list of success rates for the events.
    """
    # Filter out rows where starting time is in the future
    df['starting_time'] = pd.to_datetime(df['starting_time'], errors='coerce')
    df_now = df[df['starting_time'] < datetime.now()].copy()

    # Filter out rows with missing or invalid ticket price information
    df_now['ticket_price'] = df_now['ticket_price'].fillna('')
    df_now['remain_prices'] = df_now['remain_prices'].fillna('')
    filtered_df = df_now[
        (df_now['ticket_price'] != 'No information available') &
        (df_now['ticket_price'] != '0.0')
    ].copy()

    # Process ticket prices and remaining prices into lists
    filtered_df['ticket_price_list'] = filtered_df['ticket_price'].str.split(', ').apply(
        lambda x: [float(price) for price in x if price.replace('.', '', 1).isdigit()]
    )
    filtered_df['remain_prices_list'] = filtered_df['remain_prices'].str.split(', ').apply(
        lambda x: [float(price) if price != 'SOLD OUT' else 0.0 for price in x]
    )

    # Calculate mean ticket prices and remaining prices
    filtered_df['mean_ticket_price'] = filtered_df['ticket_price_list'].apply(lambda x: round(np.mean(x), 2))
    filtered_df['mean_remain_price'] = filtered_df['remain_prices_list'].apply(lambda x: round(np.mean(x), 2))

    # Calculate success rate
    filtered_df['success_rate'] = filtered_df.apply(
        lambda row: abs(round(1 - (row['mean_remain_price'] / row['mean_ticket_price']), 2)), axis=1
    )

    # Return the success rates as a list
    return filtered_df


In [156]:
success_rate(df).columns

Index(['event_title', 'event_genres', 'line_up', 'place', 'starting_time',
       'finishing_time', 'venue_information', 'event_location_details',
       'event_ticket_types', 'ticket_price', 'location_identifier',
       'location_address', 'remain_prices', 'image', 'data_date', 'url',
       'Record ID', 'id', 'city', 'starting_day', 'free_entrance', 'latitud',
       'longitud', 'district', 'ticket_price_list', 'remain_prices_list',
       'mean_ticket_price', 'mean_remain_price', 'success_rate'],
      dtype='object')

In [159]:
df_success = success_rate(df)[['event_title', 'ticket_price','mean_ticket_price', 'remain_prices', 'mean_remain_price' ,'success_rate']]

In [160]:
df_success

,event_title,ticket_price,mean_ticket_price,remain_prices,mean_remain_price,success_rate
0,LUNES | Happy Mondays!,"0.0, 10.0, 12.0, 18.0, 20.0",12.00,"0.0, 10.0, 12.0, 18.0, 20.0",12.0,0.0
1,MARTES | La3 International Club!,"0.0, 10.0, 12.0, 18.0, 20.0",12.00,"0.0, 10.0, 12.0, 18.0, 20.0",12.0,0.0
2,MIÉRCOLES | Project 3! Welcome to the fraternity,"0.0, 10.0, 15.0, 20.0, 90.0",27.00,"0.0, 10.0, 15.0, 20.0, 90.0",27.0,0.0
3,Jueves 21/11 // MOOVA en LATEX,"0.0, 12.0",6.00,"0.0, 12.0",6.0,0.0
4,Bambant: Aaron Gehrig + Segu + Victor Polo / B...,"0.0, 12.0",6.00,"0.0, 12.0",6.0,0.0
...,...,...,...,...,...,...
736,Jueves 5 de diciembre // CAMPUS THURSDAY,150.0,150.00,SOLD OUT,0.0,1.0
737,Viernes 6 de diciembre,"120.0, 150.0",135.00,SOLD OUT,0.0,1.0
738,Jagger Club - Despechás & special show Magic Park,"0.0, 100.0, 120.0",73.33,SOLD OUT,0.0,1.0
739,Tardeo 6 Diciembre,"12.0, 200.0, 250.0",154.00,"12.0, 200.0, 250.0",154.0,0.0
